In [2]:
import sys
import os
import pprint
import pandas as pd

In [3]:
path_to_talla = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

In [4]:
if path_to_talla not in sys.path:
    sys.path.append(path_to_talla)

In [5]:
from FinABSA.FinABSA import ABSA

In [6]:
news = pd.read_csv('../data/newsbank_articles.csv')

In [7]:
print(news['Content'][5])

Netflix, Oracle and other big technology stocks lifted Wall Street Wednesday as their profits pile higher and excitement builds around the moneymaking prospects of artificial intelligence. The S&P 500 rose 0.6% and came close to its all-time closing high set early last month. The Dow Jones Industrial Average added 130 points, or 0.3%, and the Nasdaq composite climbed 1.3%. The gains came even though most U.S. stocks fell under the weight of another crank higher for Treasury yields in the bond market. The smaller stocks in the Russell 2000 index lost 0.6%, for example, and roughly two out of every three stocks in the S&P 500 sank. Gains for big, influential stocks were more than enough to make up for it. Netflix helped lead the way after it said live events like football games and a Mike Tyson-Jake Paul fight helped it add nearly 19 million subscribers during the latest quarter. Oracle added 6.8% following its 7.2% rise the day before, ahead of the expected announcement about Stargate, 

In [8]:
absa = ABSA()
o = absa.run_absa(
                input_str = news['Content'][5]
                )
pprint.pprint(o)

2025-10-14 13:50:02,597 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
{'Dow Jones Industrial Average': {'classification_output': 'POSITIVE',
                                  'logits': {'negative': 2.559057429607492e-05,
                                             'neutral': 0.020032662898302078,
                                             'positive': 0.979941725730896}},
 'Nasdaq': {'classification_output': 'NEUTRAL',
            'logits': {'negative': 0.0002637641446199268,
                       'neutral': 0.8772879242897034,
                       'positive': 0.12244825065135956}},
 'Netflix': {'classification_output': 'POSITIVE',
             'logits': {'negative': 2.174526161979884e-05,
                        'neutral': 0.00019207052537240088,
                        'positive': 0.9997861981391907}},
 'OpenAI': {'classification_outpu

In [9]:
o.keys()

dict_keys(['Netflix', 'Oracle', 'Dow Jones Industrial Average', 'Nasdaq', 'Treasury', 'Stargate', 'OpenAI', 'SoftBank'])

In [1]:
import torch
path_to_finentity = os.path.join(path_to_talla, 'FinEntity')

if path_to_finentity not in sys.path:
    sys.path.append(path_to_finentity)


NameError: name 'os' is not defined

In [36]:
# Now, open the file using the full path
model_file_path = os.path.join(path_to_finentity, 'model_bert_crf')
import pickle
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            # Intercept the problematic function and rewrite it to use CPU
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu', weights_only=False)
        else:
            return super().find_class(module, name)

# --- Load the model using the custom unpickler ---
print(f"Loading model from: {model_file_path}")
try:
    with open(model_file_path, 'rb') as f:
        model = CPU_Unpickler(f).load()
    
    print("Model loaded successfully using custom CPU unpickler!")

except Exception as e:
    print(f"An error occurred: {e}")

Loading model from: c:\Users\talla\FinEntity\model_bert_crf
Model loaded successfully using custom CPU unpickler!


In [2]:
import json
from sequence_aligner.dataset import PredictDatasetCRF
from sequence_aligner.containers import TraingingBatch,PredictBatch
from transformers import BertTokenizerFast
from sequence_aligner.labelset import LabelSet
from util.train import predict,predict4news
from torch import cuda
import warnings
warnings.filterwarnings('ignore')

f3 = open(path_to_finentity+'/news_data/20230105.json', 'r', encoding='utf8')
content3 = f3.read() 
raw = json.loads(content3)
data = [example for example in raw if example["content"]!= None and example["content"]!=[]]


save_list = []
tokenizer = BertTokenizerFast.from_pretrained('yiyanghkust/finbert-pretrain')
label_set = LabelSet(labels=["Neutral", "Positive", "Negative"])  # label in this dataset
model.to('cpu')

'''
for item in data:
    predict4news(model,item,device,tokenizer,label_set,save_list)
'''

ModuleNotFoundError: No module named 'sequence_aligner'

In [1]:
data

NameError: name 'data' is not defined